In [182]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn import linear_model as lm
from sklearn.feature_selection import r_regression
from sklearn.metrics import r2_score

In [183]:
import warnings
warnings.filterwarnings('ignore')

In [184]:
approval = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS-FKWVTTFtJT6u56e0bqdfoMcXvDO1DUChsJ3jQAMB2lZk2SMqVfmg7dGjclTYkYWz-Pm5lfcLPjp4/pub?output=csv')
# approval = approval.reset_index()
# approval = approval.rename({'index':'poll_id'}, axis=1)
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,tracking,approve,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp
0,Donald Trump (2nd term),Economy,Verasight,Strength in Numbers,10/24/2025,10/29/2025,1567.0,A,1.004010,1.00401,NaN,40.0,54.0,-14.0,40.05299,54.635620,-14.582630,https://www.gelliottmorris.com/p/new-poll-vote...,10/29/2025,14:00:58 31 Oct 2025
1,Donald Trump (2nd term),Immigration,Verasight,Strength in Numbers,10/24/2025,10/29/2025,1567.0,A,1.004010,1.00401,NaN,46.0,49.0,-3.0,46.03001,48.794720,-2.764709,https://www.gelliottmorris.com/p/new-poll-vote...,10/29/2025,14:01:01 31 Oct 2025
2,Donald Trump (2nd term),Cost,Verasight,Strength in Numbers,10/24/2025,10/29/2025,1567.0,A,1.004010,1.00401,NaN,35.0,61.0,-26.0,35.84896,60.378520,-24.529560,https://www.gelliottmorris.com/p/new-poll-vote...,10/29/2025,14:01:03 31 Oct 2025
3,Donald Trump (2nd term),Weak,Verasight,Strength in Numbers,10/24/2025,10/29/2025,1567.0,A,1.004010,1.00401,NaN,17.0,10.0,7.0,15.90488,9.610785,6.294095,https://www.gelliottmorris.com/p/new-poll-vote...,10/29/2025,14:00:55 31 Oct 2025
4,Donald Trump (2nd term),Trade,Verasight,Strength in Numbers,10/24/2025,10/29/2025,1567.0,A,1.004010,1.00401,NaN,39.0,55.0,-16.0,39.23451,55.033530,-15.799020,https://www.gelliottmorris.com/p/new-poll-vote...,10/29/2025,14:01:05 31 Oct 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2952,Donald Trump (2nd term),Weak,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.00000,NaN,15.0,8.0,7.0,12.94803,6.989388,5.958642,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,14:00:55 31 Oct 2025
2953,Donald Trump (2nd term),Trade,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.996709,0.00000,NaN,38.0,40.0,-2.0,40.49933,40.792240,-0.292911,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,14:01:05 31 Oct 2025
2954,Donald Trump (2nd term),Adults,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.00000,NaN,47.0,41.0,6.0,47.80711,40.291460,7.515652,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,14:00:48 31 Oct 2025
2955,Donald Trump (2nd term),All polls,InsiderAdvantage,NaN,1/20/2025,1/20/2025,800.0,RV,1.093180,0.00000,NaN,56.0,39.0,17.0,52.18683,42.063030,10.123800,https://insideradvantage.com/first-insideradva...,1/20/2025,14:00:37 31 Oct 2025


In [185]:
# approval[approval['pollster']=='YouGov'][approval['subgroup']=='All polls']

In [186]:
approval = approval[~approval['subgroup'].isin(['Weak', 'Strong', 'Economy', 'Cost', 'Immigration', 'Trade', 'Adults', 'All polls'])]

In [187]:
approval.to_csv('trump_approval_polls_RV.csv')

In [188]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'enddate', 'samplesize', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp'], dtype=object)

In [189]:
approval.shape

(517, 20)

In [190]:
# dates = approval['Dates'].str.extract(r'(\d{1,2}\/\d{1,2}) \- (\d{1,2}\/\d{1,2}), (\d{4})')
# dates

In [191]:
# dates['end_date'] = dates[1] + '/' + dates[2]
# dates['end_date'] = pd.to_datetime(dates['end_date'])
# dates

In [192]:
# approval['Dates'] = dates['end_date']
# approval.head()

In [193]:
# samples = approval['Sample'].str.extract(r'[0-9,]{3,5} (\w{1,2})@@(\d{3,4})')
# samples

In [194]:
# approval['population'] = samples[0]
# approval['sample_size'] = samples[1].astype(int)
# approval.head()

In [195]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [196]:
ratings['Pollster'].values

array(['AtlasIntel', 'Marquette University Law School',
       'The Washington Post', 'Research & Polling Inc.',
       'The New York Times/Siena College', 'Cygnal', 'SurveyUSA',
       'Siena College', 'Beacon Research/Shaw & Co. Research',
       'Alaska Survey Research', 'OnMessage Inc.', 'TIPP Insights',
       'Emerson College', 'Landmark Communications',
       'Monmouth University', 'Data Orbital',
       'University of North Florida', 'MassINC Polling Group',
       'Marist College', 'Public Policy Institute of California',
       'Public Policy Polling', 'University of California Berkeley',
       'CBS News/The New York Times', 'Christopher Newport University',
       'Mason-Dixon Polling & Strategy', 'Strategic Vision LLC',
       'Suffolk University', 'Stockton University',
       'Field Research Corp. (Field Poll)', 'Quinnipiac University',
       'Los Angeles Times',
       'Hart Research Associates/Public Opinion Strategies',
       'Public Opinion Strategies', 'Susquehan

In [197]:
ratings[ratings['Pollster'].str.contains('North Star')]#.sum()

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
75,North Star Opinion Research,B/C@@8,0.043339,R+0.63@@-0.62697512,13,R+
425,Univision/Latino Decisions/North Star Opinion ...,B/C@@8,0.830409,D+1.06@@1.0564657,7,D+


In [198]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [199]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(517, 20)

In [200]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='left')
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),Voters,RMG Research,Napolitan News Service,10/22/2025,10/30/2025,3000.0,RV,0.948596,0.890399,...,49.0,-1.0,44.55356,51.56797,-7.014412,https://napolitannews.org/jobapproval,10/30/2025,14:00:44 31 Oct 2025,RMG Research,0.278023
1,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/24/2025,10/30/2025,1500.0,LV,0.842307,0.842307,...,53.0,-8.0,42.27254,54.58352,-12.310980,https://www.rasmussenreports.com/public_conten...,10/30/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015
2,Donald Trump (2nd term),Voters,YouGov,Economist,10/24/2025,10/27/2025,1476.0,RV,0.765482,0.719489,...,55.0,-12.0,43.74231,53.90477,-10.162460,https://d3nkl3psvxxpe9.cloudfront.net/document...,10/27/2025,14:00:44 31 Oct 2025,YouGov,0.047635
3,Donald Trump (2nd term),Voters,Morning Consult,NaN,10/24/2025,10/26/2025,2202.0,RV,0.769120,0.674331,...,51.0,-5.0,45.43188,51.05552,-5.623638,https://pro.morningconsult.com/trackers/donald...,10/26/2025,14:00:44 31 Oct 2025,Morning Consult,0.584994
4,Donald Trump (2nd term),Voters,Quinnipiac University,NaN,10/16/2025,10/20/2025,1327.0,RV,1.384868,0.672648,...,54.0,-14.0,42.97209,52.89769,-9.925598,https://poll.qu.edu/poll-release?releaseid=3938,10/20/2025,14:00:44 31 Oct 2025,Quinnipiac University,-0.290484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,Donald Trump (2nd term),Voters,Morning Consult,NaN,1/24/2025,1/26/2025,2302.0,RV,0.795909,0.000000,...,44.0,8.0,51.43188,44.05552,7.376362,https://pro.morningconsult.com/trackers/donald...,1/26/2025,14:00:44 31 Oct 2025,Morning Consult,0.584994
513,Donald Trump (2nd term),Voters,Echelon Insights,NaN,1/22/2025,1/24/2025,1024.0,LV,1.211361,0.000000,...,43.0,8.0,50.53209,42.85423,7.677860,https://echeloninsights.com/in-the-news/dec-20...,1/24/2025,14:00:44 31 Oct 2025,Echelon Insights,-0.216672
514,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500.0,LV,0.213379,0.000000,...,43.0,9.0,49.27254,44.58352,4.689022,https://www.rasmussenreports.com/public_conten...,1/24/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015
515,Donald Trump (2nd term),Voters,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.0,18.0,53.55356,41.56797,11.985590,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,14:00:44 31 Oct 2025,RMG Research,0.278023


In [201]:
approval.shape

(517, 22)

In [202]:
approval['Pollster'].value_counts()

Pollster
Rasmussen Reports                                     198
YouGov                                                 49
Morning Consult                                        40
RMG Research                                           39
Global Strategy Group/GBAO (Navigator Research)        15
J.L. Partners                                          15
Echelon Insights                                       12
Quinnipiac University                                  10
Harris Insights & Analytics                            10
American Research Group                                 9
Emerson College                                         9
John Zogby Strategies                                   8
Clarity Campaign Labs                                   7
McLaughlin & Associates                                 6
Marquette University Law School                         5
SSRS                                                    5
Ipsos                                                   5
Beaco

In [203]:
# approval[~approval['tracking'].isna()]

In [204]:
approval['tracking'].value_counts()

tracking
True    198
Name: count, dtype: int64

In [205]:
# full_approval = approval.copy()

In [206]:
approval = approval[(~approval['samplesize'].isna())]
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),Voters,RMG Research,Napolitan News Service,10/22/2025,10/30/2025,3000.0,RV,0.948596,0.890399,...,49.0,-1.0,44.55356,51.56797,-7.014412,https://napolitannews.org/jobapproval,10/30/2025,14:00:44 31 Oct 2025,RMG Research,0.278023
1,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/24/2025,10/30/2025,1500.0,LV,0.842307,0.842307,...,53.0,-8.0,42.27254,54.58352,-12.310980,https://www.rasmussenreports.com/public_conten...,10/30/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015
2,Donald Trump (2nd term),Voters,YouGov,Economist,10/24/2025,10/27/2025,1476.0,RV,0.765482,0.719489,...,55.0,-12.0,43.74231,53.90477,-10.162460,https://d3nkl3psvxxpe9.cloudfront.net/document...,10/27/2025,14:00:44 31 Oct 2025,YouGov,0.047635
3,Donald Trump (2nd term),Voters,Morning Consult,NaN,10/24/2025,10/26/2025,2202.0,RV,0.769120,0.674331,...,51.0,-5.0,45.43188,51.05552,-5.623638,https://pro.morningconsult.com/trackers/donald...,10/26/2025,14:00:44 31 Oct 2025,Morning Consult,0.584994
4,Donald Trump (2nd term),Voters,Quinnipiac University,NaN,10/16/2025,10/20/2025,1327.0,RV,1.384868,0.672648,...,54.0,-14.0,42.97209,52.89769,-9.925598,https://poll.qu.edu/poll-release?releaseid=3938,10/20/2025,14:00:44 31 Oct 2025,Quinnipiac University,-0.290484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,Donald Trump (2nd term),Voters,Morning Consult,NaN,1/24/2025,1/26/2025,2302.0,RV,0.795909,0.000000,...,44.0,8.0,51.43188,44.05552,7.376362,https://pro.morningconsult.com/trackers/donald...,1/26/2025,14:00:44 31 Oct 2025,Morning Consult,0.584994
513,Donald Trump (2nd term),Voters,Echelon Insights,NaN,1/22/2025,1/24/2025,1024.0,LV,1.211361,0.000000,...,43.0,8.0,50.53209,42.85423,7.677860,https://echeloninsights.com/in-the-news/dec-20...,1/24/2025,14:00:44 31 Oct 2025,Echelon Insights,-0.216672
514,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500.0,LV,0.213379,0.000000,...,43.0,9.0,49.27254,44.58352,4.689022,https://www.rasmussenreports.com/public_conten...,1/24/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015
515,Donald Trump (2nd term),Voters,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.0,18.0,53.55356,41.56797,11.985590,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,14:00:44 31 Oct 2025,RMG Research,0.278023


In [207]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [208]:
# house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+\d.\d\d.*@@([\d\.\+\-]+)')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,-1.999072
1,D,1.017374
2,D,0.632896
3,D,0.791334
4,R,-0.597041
...,...,...
93,R,-2.443137
94,R,-0.446172
95,R,-3.234717
96,R,-4.939213


In [209]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[1]
house_eff

,0,1,house_effect
0,-1,-1.999072,-1.999072
1,1,1.017374,1.017374
2,1,0.632896,0.632896
3,1,0.791334,0.791334
4,-1,-0.597041,-0.597041
...,...,...,...
93,-1,-2.443137,-2.443137
94,-1,-0.446172,-0.446172
95,-1,-3.234717,-3.234717
96,-1,-4.939213,-4.939213


In [210]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-1.999072
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.017374
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.632896
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.791334
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.597041
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.443137
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.446172
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.234717


In [211]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['approve_adj'] = df['approve'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['disapprove_adj'] = df['disapprove'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['approve_adj'] = df['approve']
        df['disapprove_adj'] = df['disapprove']
    return df

In [212]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj
1,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/24/2025,10/30/2025,1500.0,LV,0.842307,0.842307,...,42.27254,54.58352,-12.310980,https://www.rasmussenreports.com/public_conten...,10/30/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,44.361664,53.638336
19,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/17/2025,10/23/2025,1500.0,LV,0.324773,0.184586,...,44.27254,52.58352,-8.310978,https://www.rasmussenreports.com/public_conten...,10/23/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,46.361664,51.638336
25,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/23/2025,10/29/2025,1500.0,LV,0.117946,0.110781,...,42.27254,54.58352,-12.310980,https://www.rasmussenreports.com/public_conten...,10/29/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,44.361664,53.638336
26,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/20/2025,10/24/2025,1500.0,LV,0.155384,0.107712,...,43.27254,53.58352,-10.310980,https://www.rasmussenreports.com/public_conten...,10/24/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,45.361664,52.638336
27,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/22/2025,10/28/2025,1500.0,LV,0.115884,0.101773,...,42.27254,54.58352,-12.310980,https://www.rasmussenreports.com/public_conten...,10/28/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,44.361664,53.638336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,Donald Trump (2nd term),Voters,Fabrizio Ward,Pinpoint Policy Institute,5/15/2025,5/19/2025,800.0,RV,1.129986,0.000000,...,46.16771,49.43650,-3.268791,https://pinpointpolicyinstitute.org/wp-content...,5/19/2025,14:00:44 31 Oct 2025,NaN,NaN,46.000000,49.000000
8,Donald Trump (2nd term),Voters,Hart Research,Senate Majority PAC,10/17/2025,10/19/2025,1221.0,LV,1.177471,0.574184,...,44.89692,54.29254,-9.395622,https://senatemajority.com/wp-content/uploads/...,10/19/2025,14:00:44 31 Oct 2025,NaN,NaN,45.000000,55.000000
270,Donald Trump (2nd term),Voters,AP-NORC,NaN,5/1/2025,5/5/2025,1175.0,LV,1.145508,0.000000,...,42.17234,55.37215,-13.199810,https://apnorc.org/wp-content/uploads/2025/05/...,5/5/2025,14:00:44 31 Oct 2025,NaN,NaN,41.000000,57.000000
174,Donald Trump (2nd term),Voters,High Point University,NaN,7/18/2025,7/21/2025,847.0,RV,0.985411,0.000000,...,42.05438,51.41337,-9.358990,https://www.highpoint.edu/src/files/2023/08/hp...,7/21/2025,14:00:44 31 Oct 2025,High Point University,0.411680,41.000000,51.000000


In [213]:
approval.shape

(516, 22)

In [214]:
approval = approval_adj.copy()
approval.shape

(516, 24)

In [215]:
approval['poll_id'] = pd.factorize(approval['pollster'] + approval['enddate'] + approval['sponsors'].fillna('none'))[0]
approval.head(7)

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj,poll_id
1,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/24/2025,10/30/2025,1500.0,LV,0.842307,0.842307,...,54.58352,-12.310980,https://www.rasmussenreports.com/public_conten...,10/30/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,44.361664,53.638336,0
19,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/17/2025,10/23/2025,1500.0,LV,0.324773,0.184586,...,52.58352,-8.310978,https://www.rasmussenreports.com/public_conten...,10/23/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,46.361664,51.638336,1
25,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/23/2025,10/29/2025,1500.0,LV,0.117946,0.110781,...,54.58352,-12.310980,https://www.rasmussenreports.com/public_conten...,10/29/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,44.361664,53.638336,2
26,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/20/2025,10/24/2025,1500.0,LV,0.155384,0.107712,...,53.58352,-10.310980,https://www.rasmussenreports.com/public_conten...,10/24/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,45.361664,52.638336,3
27,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/22/2025,10/28/2025,1500.0,LV,0.115884,0.101773,...,54.58352,-12.310980,https://www.rasmussenreports.com/public_conten...,10/28/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,44.361664,53.638336,4
28,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/21/2025,10/27/2025,1500.0,LV,0.114323,0.093368,...,54.58352,-12.310980,https://www.rasmussenreports.com/public_conten...,10/27/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,44.361664,53.638336,5
30,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/10/2025,10/16/2025,1500.0,LV,0.319266,0.075774,...,51.58352,-5.310978,https://www.rasmussenreports.com/public_conten...,10/16/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,48.361664,50.638336,6


In [216]:
approval['pollster_id'] = pd.factorize(approval['pollster'] + approval['sponsors'].fillna('none'))[0]

In [217]:
approval['population'] = approval['population'].str.strip(' ')

In [218]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id', keep=False)]
    ndup = data[~data.duplicated(subset='poll_id', keep=False)]
    # print('dup', dup, '\n ndup', ndup)
    dup_a = dup[dup['population'].isin(['RV'])]
    # print('dup_a', dup_a)
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id', keep=False)]
    ndup = df[~df.duplicated(subset='poll_id', keep=False)]
    dup_lv = dup[dup['population'].isin(['LV'])]
    # print('dup_rv', dup_lv, 'ndup', ndup)
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [219]:
approval = approval.rename({'enddate':'end_date', 'samplesize':'sample_size', 'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [220]:
approval['population'].value_counts().index.values

array(['RV', 'LV', 'V'], dtype=object)

In [221]:
approval.shape, pipeline(approval).shape

((516, 26), (514, 26))

In [222]:
pipeline(approval)

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id,pollster_id
1,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/24/2025,2025-10-30,1500.0,LV,0.842307,0.842307,...,-12.310980,https://www.rasmussenreports.com/public_conten...,10/30/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,44.361664,53.638336,0,0
19,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/17/2025,2025-10-23,1500.0,LV,0.324773,0.184586,...,-8.310978,https://www.rasmussenreports.com/public_conten...,10/23/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,46.361664,51.638336,1,0
25,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/23/2025,2025-10-29,1500.0,LV,0.117946,0.110781,...,-12.310980,https://www.rasmussenreports.com/public_conten...,10/29/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,44.361664,53.638336,2,0
26,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/20/2025,2025-10-24,1500.0,LV,0.155384,0.107712,...,-10.310980,https://www.rasmussenreports.com/public_conten...,10/24/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,45.361664,52.638336,3,0
27,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,10/22/2025,2025-10-28,1500.0,LV,0.115884,0.101773,...,-12.310980,https://www.rasmussenreports.com/public_conten...,10/28/2025,14:00:44 31 Oct 2025,Rasmussen Reports,0.087015,44.361664,53.638336,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,Donald Trump (2nd term),Voters,Fabrizio Ward,Pinpoint Policy Institute,5/15/2025,2025-05-19,800.0,RV,1.129986,0.000000,...,-3.268791,https://pinpointpolicyinstitute.org/wp-content...,5/19/2025,14:00:44 31 Oct 2025,NaN,NaN,46.000000,49.000000,510,63
8,Donald Trump (2nd term),Voters,Hart Research,Senate Majority PAC,10/17/2025,2025-10-19,1221.0,LV,1.177471,0.574184,...,-9.395622,https://senatemajority.com/wp-content/uploads/...,10/19/2025,14:00:44 31 Oct 2025,NaN,NaN,45.000000,55.000000,511,64
270,Donald Trump (2nd term),Voters,AP-NORC,NaN,5/1/2025,2025-05-05,1175.0,LV,1.145508,0.000000,...,-13.199810,https://apnorc.org/wp-content/uploads/2025/05/...,5/5/2025,14:00:44 31 Oct 2025,NaN,NaN,41.000000,57.000000,512,65
174,Donald Trump (2nd term),Voters,High Point University,NaN,7/18/2025,2025-07-21,847.0,RV,0.985411,0.000000,...,-9.358990,https://www.highpoint.edu/src/files/2023/08/hp...,7/21/2025,14:00:44 31 Oct 2025,High Point University,0.411680,41.000000,51.000000,513,66


In [223]:
# approval[approval['pollster']=='Harris Insights & Analytics']

In [224]:
full_approval = approval.copy()

In [225]:
approval = approval[approval['tracking'].isna()]

In [226]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'TIPP Insights', 'Trafalgar Group',
       'Trafalgar Group/Insider Advantage'], dtype=object)

In [227]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id,pollster_id
2,Donald Trump (2nd term),Voters,YouGov,Economist,10/24/2025,10/27/2025,1476.0,RV,0.765482,0.719489,...,-10.16246,https://d3nkl3psvxxpe9.cloudfront.net/document...,10/27/2025,14:00:44 31 Oct 2025,YouGov,0.047635,43.249587,54.750413,197,3
7,Donald Trump (2nd term),Voters,YouGov,Yahoo News,10/23/2025,10/27/2025,1197.0,RV,0.685971,0.603411,...,-10.16246,https://www.scribd.com/document/939641649/Yaho...,10/27/2025,14:00:44 31 Oct 2025,YouGov,0.047635,43.249587,54.750413,198,4
14,Donald Trump (2nd term),Voters,YouGov,Economist,10/17/2025,10/20/2025,1448.0,RV,0.713792,0.361072,...,-10.16246,https://d3nkl3psvxxpe9.cloudfront.net/document...,10/20/2025,14:00:44 31 Oct 2025,YouGov,0.047635,43.249587,54.750413,199,3
21,Donald Trump (2nd term),Voters,YouGov,Economist,10/10/2025,10/13/2025,1467.0,RV,0.726330,0.150881,...,-8.16246,https://d3nkl3psvxxpe9.cloudfront.net/document...,10/13/2025,14:00:44 31 Oct 2025,YouGov,0.047635,44.249587,53.750413,200,3
33,Donald Trump (2nd term),Voters,YouGov,Economist,10/4/2025,10/6/2025,1490.0,RV,0.700719,0.047927,...,-10.16246,https://d3nkl3psvxxpe9.cloudfront.net/document...,10/6/2025,14:00:44 31 Oct 2025,YouGov,0.047635,43.249587,54.750413,201,3


In [228]:
approval[approval['pollster']=='Civiqs']# [['approve', 'disapprove', 'approve_adj', 'disapprove_adj']]

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id,pollster_id
290,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,5/17/2025,5/20/2025,1018.0,RV,1.120966,0.0,...,-4.276749,https://civiqs.com/documents/Civiqs_DailyKos_b...,5/20/2025,14:00:44 31 Oct 2025,Civiqs,0.158562,47.860928,51.139072,477,43
364,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,4/12/2025,4/15/2025,1124.0,RV,1.149724,0.0,...,-6.276749,https://civiqs.com/reports/2025/04/17/report-a...,4/15/2025,14:00:44 31 Oct 2025,Civiqs,0.158562,46.860928,52.139072,478,43
398,Donald Trump (2nd term),Voters,Civiqs,Daily Kos,2/28/2025,3/3/2025,1031.0,RV,1.124632,0.0,...,-3.276749,https://civiqs.com/reports/2025/03/04/report-a...,3/3/2025,14:00:44 31 Oct 2025,Civiqs,0.158562,48.860928,51.139072,479,43


In [229]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'end_date', 'sample_size', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp', 'Pollster', 'pred_plus_minus', 'approve_adj',
       'disapprove_adj', 'poll_id', 'pollster_id'], dtype=object)

In [230]:
approval_polls = approval[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'approve_adj', 'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id', 'pollster_id']]
approval_polls

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,pollster_id
2,YouGov,Economist,10/24/2025,10/27/2025,1476.0,RV,43.0,55.0,43.249587,54.750413,-12.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,197,3
7,YouGov,Yahoo News,10/23/2025,10/27/2025,1197.0,RV,43.0,55.0,43.249587,54.750413,-12.0,https://www.scribd.com/document/939641649/Yaho...,0.047635,198,4
14,YouGov,Economist,10/17/2025,10/20/2025,1448.0,RV,43.0,55.0,43.249587,54.750413,-12.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,199,3
21,YouGov,Economist,10/10/2025,10/13/2025,1467.0,RV,44.0,54.0,44.249587,53.750413,-10.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,200,3
33,YouGov,Economist,10/4/2025,10/6/2025,1490.0,RV,43.0,55.0,43.249587,54.750413,-12.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,201,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,Fabrizio Ward,Pinpoint Policy Institute,5/15/2025,5/19/2025,800.0,RV,46.0,49.0,46.000000,49.000000,-3.0,https://pinpointpolicyinstitute.org/wp-content...,NaN,510,63
8,Hart Research,Senate Majority PAC,10/17/2025,10/19/2025,1221.0,LV,45.0,55.0,45.000000,55.000000,-10.0,https://senatemajority.com/wp-content/uploads/...,NaN,511,64
270,AP-NORC,NaN,5/1/2025,5/5/2025,1175.0,LV,41.0,57.0,41.000000,57.000000,-16.0,https://apnorc.org/wp-content/uploads/2025/05/...,NaN,512,65
174,High Point University,NaN,7/18/2025,7/21/2025,847.0,RV,41.0,51.0,41.000000,51.000000,-10.0,https://www.highpoint.edu/src/files/2023/08/hp...,0.411680,513,66


In [231]:
approval_polls.shape

(307, 15)

In [232]:
def zone_flood_checker(data, pid, date):
    """
    Check whether there are multiple polls from the same pollster + same sponsor (if applicable) within the last two weeks, 
    which could indicate "flooding the zone." Return number of polls from same pollster/sponsor in last two weeks.

    :param data: Polls dataframe.
    :param pid: Pollster ID, specified in the polls dataframe as the column pollster_id.
    :param date: Date at which specified poll is conducted. Corresponds to column end_date in polls dataframe.
    """
    thres = date - pd.Timedelta(weeks=2)
    df = data[ (data['pollster_id'] == pid) & (pd.to_datetime(data['end_date']) >= thres) ]
    return max(df.shape[0], 1)

In [233]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    exp_weights = 0.1**(delta/(30))
    state_pivot['time_weights'] =  0 * linear_weights + 1 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if np.isnan(rel_qual): # For pollsters without a quality rating from the Silver Bulletin
            return 0.2
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'RV':
            return 0.8
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

   # Multiple polls in short window (aka "flooding the zone")
    def zone_flood_weight(pid, date):
        num_polls = zone_flood_checker(approval_polls, pid, date)
        return 1 / np.sqrt(num_polls)
    state_pivot['zone_flood_weight'] = state_pivot[['pollster_id', 'end_date']].apply(
        lambda x: zone_flood_weight(x['pollster_id'], x['end_date']),
        axis=1
    )

    # Gather the weights together
    state_pivot['total_weights'] = (state_pivot['sample_size_weights'] * state_pivot['time_weights'] * 
                                    state_pivot['quality_weights'] * state_pivot['population_weights'] * state_pivot['zone_flood_weight'])
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [234]:
approval_polls['sample_size'].isna().any()

False

In [235]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,...,url,pred_plus_minus,poll_id,pollster_id,sample_size_weights,time_weights,quality_weights,population_weights,zone_flood_weight,total_weights
2,YouGov,Economist,10/24/2025,2025-10-27,1476.0,RV,43.0,55.0,43.249587,54.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,197,3,0.003205,7.356423e-01,0.964021,0.8,0.577350,6.443145e-02
7,YouGov,Yahoo News,10/23/2025,2025-10-27,1197.0,RV,43.0,55.0,43.249587,54.750413,...,https://www.scribd.com/document/939641649/Yaho...,0.047635,198,4,0.002886,7.356423e-01,0.964021,0.8,1.000000,1.004992e-01
14,YouGov,Economist,10/17/2025,2025-10-20,1448.0,RV,43.0,55.0,43.249587,54.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,199,3,0.003174,4.298662e-01,0.964021,0.8,0.500000,3.229507e-02
21,YouGov,Economist,10/10/2025,2025-10-13,1467.0,RV,44.0,54.0,44.249587,53.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,200,3,0.003195,2.511886e-01,0.964021,0.8,0.447214,1.698943e-02
33,YouGov,Economist,10/4/2025,2025-10-06,1490.0,RV,43.0,55.0,43.249587,54.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,201,3,0.003220,1.467799e-01,0.964021,0.8,0.408248,9.133408e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,Fabrizio Ward,Pinpoint Policy Institute,5/15/2025,2025-05-19,800.0,RV,46.0,49.0,46.000000,49.000000,...,https://pinpointpolicyinstitute.org/wp-content...,NaN,510,63,0.002359,3.162278e-06,0.200000,0.8,1.000000,7.327186e-08
8,Hart Research,Senate Majority PAC,10/17/2025,2025-10-19,1221.0,LV,45.0,55.0,45.000000,55.000000,...,https://senatemajority.com/wp-content/uploads/...,NaN,511,64,0.002915,3.981072e-01,0.200000,1.0,1.000000,1.424492e-02
270,AP-NORC,NaN,5/1/2025,2025-05-05,1175.0,LV,41.0,57.0,41.000000,57.000000,...,https://apnorc.org/wp-content/uploads/2025/05/...,NaN,512,65,0.002859,1.079775e-06,0.200000,1.0,1.000000,3.790133e-08
174,High Point University,NaN,7/18/2025,2025-07-21,847.0,RV,41.0,51.0,41.000000,51.000000,...,https://www.highpoint.edu/src/files/2023/08/hp...,0.411680,513,66,0.002428,3.981072e-04,0.728566,0.8,1.000000,3.457585e-05


In [236]:
avgs_pipeline(approval_polls, datetime.datetime.today()).shape

(307, 21)

In [237]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['approve_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['disapprove_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [238]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Approve,Disapprove
0,2025-01-23,56.180465,39.239431
1,2025-01-24,53.709213,41.033346
2,2025-01-25,53.709213,41.033346
3,2025-01-26,53.637667,41.175202
4,2025-01-27,50.961236,42.182946
...,...,...,...
277,2025-10-27,45.045540,52.046811
278,2025-10-28,45.045540,52.046811
279,2025-10-29,45.045540,52.046811
280,2025-10-30,45.338212,51.745194


In [239]:
polls_df.shape

(307, 21)

In [240]:
polls_df.head(3)

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,...,url,pred_plus_minus,poll_id,pollster_id,sample_size_weights,time_weights,quality_weights,population_weights,zone_flood_weight,total_weights
2,YouGov,Economist,10/24/2025,2025-10-27,1476.0,RV,43.0,55.0,43.249587,54.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,197,3,0.003205,0.735642,0.964021,0.8,0.57735,0.064431
7,YouGov,Yahoo News,10/23/2025,2025-10-27,1197.0,RV,43.0,55.0,43.249587,54.750413,...,https://www.scribd.com/document/939641649/Yaho...,0.047635,198,4,0.002886,0.735642,0.964021,0.8,1.00000,0.100499
14,YouGov,Economist,10/17/2025,2025-10-20,1448.0,RV,43.0,55.0,43.249587,54.750413,...,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,199,3,0.003174,0.429866,0.964021,0.8,0.50000,0.032295


In [241]:
polls_df['net_adj'] = polls_df['approve_adj'] - polls_df['disapprove_adj']

In [242]:
display_df = polls_df[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'net',
                      'total_weights', 'url']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 'total_weights':'Weight', 'startdate':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population']
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df['Pollster'] = '[' + display_df['Pollster'] + '](' + display_df['URL'] + ')'
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net
0,"<a href=""https://napolitannews.org/jobapproval...",2025-10-22,2025-10-30,3000 RV,9.686102e-02,48.0,49.0,-1.0
1,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-10-24,2025-10-27,1476 RV,6.443145e-02,43.0,55.0,-12.0
2,"<a href=""https://navigatorresearch.org/wp-cont...",2025-10-23,2025-10-27,1000 RV,1.100267e-02,41.0,57.0,-16.0
3,"<a href=""https://www.scribd.com/document/93964...",2025-10-23,2025-10-27,1197 RV,1.004992e-01,43.0,55.0,-12.0
4,"<a href=""https://pro.morningconsult.com/tracke...",2025-10-24,2025-10-26,2202 RV,1.512076e-02,46.0,51.0,-5.0
...,...,...,...,...,...,...,...,...
302,"<a href=""https://poll.qu.edu/poll-release?rele...",2025-01-23,2025-01-27,1019 RV,3.021056e-11,46.0,43.0,3.0
303,"<a href=""https://pro.morningconsult.com/tracke...",2025-01-24,2025-01-26,2302 RV,3.363165e-12,52.0,44.0,8.0
304,"<a href=""https://echeloninsights.com/in-the-ne...",2025-01-22,2025-01-24,1024 LV,2.708798e-11,51.0,43.0,8.0
305,"<a href=""https://x.com/ScottWRasmussen/status/...",2025-01-20,2025-01-23,3000 RV,1.246936e-11,57.0,39.0,18.0


In [243]:
display_df.to_csv('polls_display_table_RV.csv')

In [244]:
approval_std

,Date,Approve,Disapprove
0,2025-01-23,0.754336,0.260794
1,2025-01-24,2.644376,1.887744
2,2025-01-25,2.644376,1.887744
3,2025-01-26,2.586147,1.924601
4,2025-01-27,3.835920,2.053782
...,...,...,...
277,2025-10-27,3.273729,3.282225
278,2025-10-28,3.273729,3.282225
279,2025-10-29,3.273729,3.282225
280,2025-10-30,3.236955,3.252346


In [245]:
approval_avg['Net'] = approval_avg['Approve'] - approval_avg['Disapprove']
approval_avg

,Date,Approve,Disapprove,Net
0,2025-01-23,56.180465,39.239431,16.941034
1,2025-01-24,53.709213,41.033346,12.675868
2,2025-01-25,53.709213,41.033346,12.675868
3,2025-01-26,53.637667,41.175202,12.462464
4,2025-01-27,50.961236,42.182946,8.778291
...,...,...,...,...
277,2025-10-27,45.045540,52.046811,-7.001270
278,2025-10-28,45.045540,52.046811,-7.001270
279,2025-10-29,45.045540,52.046811,-7.001270
280,2025-10-30,45.338212,51.745194,-6.406982


In [246]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Approve_x'] + 1.96*approval_trendline['Approve_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Approve_x'] - 1.96*approval_trendline['Approve_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Disapprove_x'] + 1.96*approval_trendline['Disapprove_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Disapprove_x'] - 1.96*approval_trendline['Disapprove_y']
approval_trendline = approval_trendline.rename({
    'Approve_x':'approve_mean',
    'Disapprove_x':'disapprove_mean',
}, axis='columns').drop(['Approve_y', 'Disapprove_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-23,56.180465,39.239431,57.658962,54.701967,39.750587,38.728275
1,2025-01-24,53.709213,41.033346,58.892191,48.526235,44.733325,37.333366
2,2025-01-25,53.709213,41.033346,58.892191,48.526235,44.733325,37.333366
3,2025-01-26,53.637667,41.175202,58.706514,48.568819,44.947420,37.402985
4,2025-01-27,50.961236,42.182946,58.479639,43.442834,46.208359,38.157532
...,...,...,...,...,...,...,...
277,2025-10-27,45.045540,52.046811,51.462049,38.629032,58.479972,45.613649
278,2025-10-28,45.045540,52.046811,51.462049,38.629032,58.479972,45.613649
279,2025-10-29,45.045540,52.046811,51.462049,38.629032,58.479972,45.613649
280,2025-10-30,45.338212,51.745194,51.682645,38.993780,58.119792,45.370595


In [247]:
# Export approval ratings
approval_trendline.to_csv('approval_stats_RV.csv')

In [248]:
polls_df.shape

(307, 22)

In [249]:
line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
line.update_traces(hovertemplate=None)
line.update_layout(hovermode='x unified')
# scatter.update_traces(hovertemplate=None, hoverinfo='skip')
fig_approve_CI = go.Figure([
    go.Scatter(
        name='Approve CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Approve CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])
fig_disapprove_CI = go.Figure([
    go.Scatter(
        name='Disapprove CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Disapprove CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])

fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
fig.update_layout(
    title='President Trump Approval Rating',
    xaxis_title='Date',
    yaxis_title='%',
    legend_title='Legend',
    hovermode='x unified'
)
fig.show()

In [250]:
line = px.line(approval_avg, x='Date', y=['Net'])
scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net'])
                     #trendline='lowess', trendline_options=dict(frac=0.2))
fig = go.Figure(data=line.data + scatter.data)
fig.add_hline(y=0, line_dash='dash')
fig.show()

In [251]:
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net', 'net_adj'])
# fig = go.Figure(data=scatter.data)
# fig.add_hline(y=0, line_dash='dash')
# fig.show()

In [252]:
def net_col(df: pd.DataFrame, min_date: datetime.date):
#     df_copy = df.set_index(['Date'])
#     return df_copy['approve_mean'] - df_copy['disapprove_mean']
    date_range = pd.date_range(start=min_date, end=datetime.datetime.today(), freq='d', inclusive='both')
    data = df.copy()
    data['net_adj'] = data['approve_adj'] - data['disapprove_adj']
    net_averages = []
    net_stdevs = []
    issue_cols = {'Date': date_range, 'net':[], 'net_std':[]}
    for date in date_range:
    #         try:
            pipelined_df = avgs_pipeline(data, date)
    #         except TypeError:
    #             print('TypeError')
    #             print(date)
            net_avg = np.sum(pipelined_df['net_adj'] * pipelined_df['total_weights'])
            net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net_adj'] - net_avg)**2))

            net_averages.append(net_avg)
            net_stdevs.append(net_std)

    issue_cols['net'].extend(net_averages)
    issue_cols[f'net_std'].extend(net_stdevs)
        
    net_averages, net_stdevs = [], []
        
    return pd.DataFrame(issue_cols)

In [253]:
net_approval = net_col(approval, datetime.date(2025, 1, 23))
net_approval

,Date,net,net_std
0,2025-01-23,16.941034,1.015130
1,2025-01-24,12.675868,4.525733
2,2025-01-25,12.675868,4.525733
3,2025-01-26,12.462464,4.483962
4,2025-01-27,8.778291,5.514618
...,...,...,...
277,2025-10-27,-7.001270,6.152043
278,2025-10-28,-7.001270,6.152043
279,2025-10-29,-7.001270,6.152043
280,2025-10-30,-6.406982,6.121661


In [254]:
net_approval['net_upper_CI'] = net_approval['net'] + 1.96*net_approval['net_std']
net_approval['net_lower_CI'] = net_approval['net'] - 1.96*net_approval['net_std']
net_approval

,Date,net,net_std,net_upper_CI,net_lower_CI
0,2025-01-23,16.941034,1.015130,18.930688,14.951380
1,2025-01-24,12.675868,4.525733,21.546305,3.805430
2,2025-01-25,12.675868,4.525733,21.546305,3.805430
3,2025-01-26,12.462464,4.483962,21.251030,3.673899
4,2025-01-27,8.778291,5.514618,19.586942,-2.030361
...,...,...,...,...,...
277,2025-10-27,-7.001270,6.152043,5.056733,-19.059274
278,2025-10-28,-7.001270,6.152043,5.056733,-19.059274
279,2025-10-29,-7.001270,6.152043,5.056733,-19.059274
280,2025-10-30,-6.406982,6.121661,5.591474,-18.405438


In [255]:
net_approval.to_csv('net_approval_RV.csv')

### Auxiliary Graph: Individual Pollsters

In [257]:
approval_polls.columns.values

array(['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size',
       'population', 'approve', 'disapprove', 'approve_adj',
       'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id',
       'pollster_id'], dtype=object)

In [258]:
appr = full_approval.copy()
appr['net_adj'] = appr['approve'] - appr['disapprove']
appr = appr[~appr['pred_plus_minus'].isna()]
pollsters = appr['pollster'].value_counts().index.values
pollsters

array(['Rasmussen Reports', 'YouGov', 'Morning Consult', 'RMG Research',
       'Global Strategy Group/GBAO (Navigator Research)', 'J.L. Partners',
       'Echelon Insights', 'Harris Insights & Analytics',
       'Quinnipiac University', 'American Research Group',
       'Emerson College', 'John Zogby Strategies',
       'Clarity Campaign Labs', 'McLaughlin & Associates', 'Ipsos',
       'Beacon Research/Shaw & Co. Research',
       'Marquette University Law School', 'InsiderAdvantage', 'SSRS',
       'Marist College', 'TIPP Insights', 'co/efficient', 'Wick',
       'Fabrizio Lee & Associates/Impact Research', 'Civiqs', 'Cygnal',
       'OH Predictive Insights', 'Blueprint Polling', 'The Winston Group',
       'Trafalgar Group', 'GBAO', 'The New York Times/Siena College',
       'Targoz Market Research', 'Public Opinion Strategies',
       'High Point University', 'Normington, Petts & Associates',
       'NewsNation', 'American Pulse Research & Polling',
       'Remington Research Grou

In [259]:
def approval_today(data: pd.DataFrame):
    # date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
#     yes_averages = []
#     no_averages = []
#     yes_stdevs = []
#     no_stdevs = []
    pipelined_df = avgs_pipeline(data, datetime.datetime.today())
    yes_avg = np.sum(pipelined_df['approve'] * pipelined_df['total_weights'])
    no_avg = np.sum(pipelined_df['disapprove'] * pipelined_df['total_weights'])
    yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve'] - yes_avg)**2))
    no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove'] - no_avg)**2))
    net_avg = np.sum(pipelined_df['net'] * pipelined_df['total_weights'])
    net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net'] - net_avg)**2))
    
    #return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [260]:
def indiv_pollster_avg(pollster: str):
    df = appr[appr['pollster'] == pollster]
    yes_avg, no_avg, yes_std, no_std, net_avg, net_std = approval_today(df)
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [261]:
def get_ppm(pollster: str):
    df = ratings[ratings['Pollster'] == pollster]
    ppm = df['Predictive Plus-Minus'].values[0]
    
    return ppm

In [262]:
appr.shape

(497, 27)

In [263]:
def get_num_polls(pollster: str):
    df = appr[appr['pollster'] == pollster]
    num_polls = df.shape[0]
    return num_polls

In [264]:
get_ppm('YouGov')

0.047635015

In [265]:
indiv_pollster_avg('YouGov')

(43.11057638662866,
 54.90888784601444,
 0.3137452440565687,
 0.30374613406288575,
 -11.798311459385781,
 0.5587992907542628)

In [266]:
indiv_pollster_avg('RMG Research')

(49.66401000040211,
 48.125499246614446,
 1.6523471228387236,
 0.9327866173345082,
 1.538510753787637,
 2.5476051543992817)

In [267]:
nets = []
net_stdevs = []
ppms = []
nums = []

for p in pollsters:
    _, _, _, _, net, net_std = indiv_pollster_avg(p)
    ppm = get_ppm(p)
    num_polls = get_num_polls(p)
    
    nets.append(net)
    net_stdevs.append(net_std)
    ppms.append(ppm)
    nums.append(num_polls)

In [268]:
# nets

In [269]:
indiv_avgs = pd.DataFrame({'Pollster': pollsters, 'Net Approval': nets, 'Net Stdev': net_stdevs, 'Predictive Plus-Minus': ppms, 'Number of Approval Polls Conducted': nums})
indiv_avgs = indiv_avgs.sort_values(['Net Approval'], ascending=True)
indiv_avgs

,Pollster,Net Approval,Net Stdev,Predictive Plus-Minus,Number of Approval Polls Conducted
9,American Research Group,-22.862693,0.501075,0.818141,9
8,Quinnipiac University,-14.166086,0.600263,-0.290484,10
4,Global Strategy Group/GBAO (Navigator Research),-13.481168,2.547909,0.804988,15
18,SSRS,-12.003751,0.094896,0.966835,4
36,NewsNation,-12.000000,0.000000,0.614459,1
1,YouGov,-11.798311,0.558799,0.047635,49
28,The Winston Group,-11.019177,0.276301,-0.136293,2
31,The New York Times/Siena College,-11.000004,0.001929,-0.763947,2
19,Marist College,-10.998852,0.047879,-0.436610,4
32,Targoz Market Research,-10.995760,0.159436,0.731655,2


In [270]:
# indiv_avgs.to_csv('indiv_pollster_avgs_RV.csv')

In [271]:
reg = lm.LinearRegression()
reg.fit(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
reg.coef_, reg.intercept_

(array([-2.42486428]), -5.490536015443106)

In [272]:
r = r_regression(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
r

array([-0.21614272])